In [ ]:
from tqdm import tqdm
import numpy as np
import torch
import copy
import math
from torch.utils.data import DataLoader
from patter import ModelFactory
from patter.data import AudioDataset, BucketingSampler, audio_seq_collate_fn
from patter.decoder import GreedyCTCDecoder
from patter.data.features import PerturbedSpectrogramFeaturizer
from patter.evaluator import validate
from patter.models import SpeechModel

In [ ]:
# load the model you're starting from
seed_model_path = "/data/users/ryan/models/deepspeech/librispeech_pretrained_patter.pt"
seed_model = ModelFactory.load(seed_model_path)

In [3]:
# specify paths of where the new model should be saved, and the train/val manifests to use in training
new_model_path = "/data/users/ryan/models/deepspeech/an4_transferred.pt"
train_manifest_path = "/home/ryan/data/patter_data/an4-jl/an4_train_manifest.jl"
val_manifest_path = "/home/ryan/data/patter_data/an4-jl/an4_val_manifest.jl"

In [4]:
# make a copy of the model
model = copy.copy(seed_model)

In [5]:
# decide on set of labels you want for the new model
# for this experiment (going from english to english, we can just reuse the set of labels from the original model)
# NB: the first label MUST represent the CTC blank label (canonically is '\xa0')
labels = seed_model.labels

# when new labels are set, tack them into the model
model.labels = labels
print("New Labels:", labels)

New Labels: ['\xa0', "'", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ']


In [6]:
# create a new output layer entirely
model.output[1] = torch.nn.Linear(in_features=800, out_features=len(labels), bias=False)

# and initialize it
for p in model.output[1].parameters():
    torch.nn.init.xavier_uniform_(p)

In [7]:
# set requires_grad = False for all but the output layers
for x, y in model.conv.named_parameters():
    y.requires_grad_(False)
for x, y in model.rnn.named_parameters():
    y.requires_grad_(False)

In [8]:
# create a featurizer based on the model's expected features, optionally specify perturbations on training data
featurizer = PerturbedSpectrogramFeaturizer.from_config(seed_model.input_cfg, perturbation_configs=None)

# load datasets for train and dev sets that you want to transfer model to
train_corpus = AudioDataset(train_manifest_path, labels, featurizer, max_duration=17.0, min_duration=1.0)
val_corpus = AudioDataset(val_manifest_path, labels, featurizer, max_duration=17.0, min_duration=1.0)

Dataset loaded with 0.70 hours. Filtered 0.00 hours.
Dataset loaded with 0.10 hours. Filtered 0.00 hours.


In [9]:
# set up data loaders
batch_size = 32
num_workers = 4
cuda = True
train_sampler = BucketingSampler(train_corpus, batch_size=batch_size)
train_loader = DataLoader(train_corpus, num_workers=num_workers, collate_fn=audio_seq_collate_fn, pin_memory=cuda, batch_sampler=train_sampler)
eval_loader = DataLoader(val_corpus, num_workers=4, collate_fn=audio_seq_collate_fn, pin_memory=cuda, batch_size=batch_size)

In [10]:
if cuda:
    model = model.cuda()

In [11]:
# set up optimizer
lr = 3e-4
momentum = 0.9
annealing = 1.01
trainable_params = set([x for x in model.parameters() if x.requires_grad])

# we typically use SGD w/ nesterov momentum and a custom learning rate schedule
#optimizer = torch.optim.SGD(trainable_params, lr=lr, momentum=momentum, nesterov=True)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1/annealing)

# Don't need a learning rate schedule if using Adam
class NoOpScheduler(object):
    def __init__(self):
        pass

    def step(self):
        pass

# but for example purposes, we'll just use Adam
optimizer = torch.optim.Adam(trainable_params, lr=lr, amsgrad=True)
scheduler = NoOpScheduler()

In [12]:
# Report the initial performance of the model on the new validation set:
err = validate(eval_loader, model, tqdm=False)
print("WER: {0:.3f}, CER: {1:.3f}".format(err.wer, err.cer))

best_wer = err.wer

WER: 102.872, CER: 277.516


In [13]:
num_epochs = 40
for epoch in range(num_epochs):
    train_sampler.shuffle()
    scheduler.step()
    
    model.train()
    train_loader = tqdm(train_loader, desc="Epoch {}".format(epoch+1), leave=False)
    for i, data in enumerate(train_loader):
        feat, target, feat_len, target_len = data
        if cuda:
            feat = feat.cuda(async=True)
        
        optimizer.zero_grad()
        
        output, output_len = model(feat, feat_len)
        loss = model.loss(output, target, output_len.squeeze(0), target_len)
        
        scalar_loss = loss.item()/feat.size(0)
        if abs(scalar_loss) == math.inf:
            scalar_loss = 0
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(trainable_params, 400)
        optimizer.step()
    model.eval()
    err = validate(eval_loader, model, tqdm=False)
    print("Epoch {0} :: WER: {1:.3f}, CER: {2:.3f}".format(epoch+1, err.wer, err.cer))
    
    if err.wer < best_wer:
        best_wer = err.wer
        torch.save(SpeechModel.serialize(model), new_model_path)

Epoch 2:   0%|          | 0/29 [00:00<?, ?it/s]         

Epoch 1 :: WER: 110.183, CER: 112.736


Epoch 2 :: WER: 97.128, CER: 58.766


Epoch 4:   0%|          | 0/29 [00:00<?, ?it/s]         

Epoch 3 :: WER: 99.217, CER: 68.868


...

Epoch 38:   0%|          | 0/29 [00:00<?, ?it/s]         

Epoch 37 :: WER: 13.708, CER: 3.970


Epoch 38 :: WER: 13.055, CER: 3.813


Epoch 39 :: WER: 12.924, CER: 3.695


Epoch 40 :: WER: 12.402, CER: 3.616


In [14]:
# reload the previously best found model
model = ModelFactory.load(new_model_path)
if cuda:
    model = model.cuda()

err = validate(eval_loader, model, tqdm=False)
print("WER: {0:.3f}, CER: {1:.3f}".format(err.wer, err.cer))

# add the rnns for additional fine tuning
for x, y in model.conv.named_parameters():
    y.requires_grad_(False)
trainable_params = set([x for x in model.parameters() if x.requires_grad])
optimizer = torch.optim.Adam(trainable_params, lr=3e-4, amsgrad=True)

WER: 12.402, CER: 3.616


In [16]:
# run another 20 epochs of training
num_epochs = 20
for epoch in range(num_epochs):
    train_sampler.shuffle()
    
    model.train()
    train_loader = tqdm(train_loader, desc="Epoch {}".format(epoch+1))
    for i, data in enumerate(train_loader):
        feat, target, feat_len, target_len = data
        if cuda:
            feat = feat.cuda(async=True)
        
        optimizer.zero_grad()
        
        output, output_len = model(feat, feat_len)
        loss = model.loss(output, target, output_len.squeeze(0), target_len)
        
        scalar_loss = loss.item()/feat.size(0)
        if abs(scalar_loss) == math.inf:
            scalar_loss = 0
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(trainable_params, 400)
        optimizer.step()
    model.eval()
    err = validate(eval_loader, model, tqdm=False)
    print("Epoch {0} :: WER: {1:.3f}, CER: {2:.3f}".format(epoch+1, err.wer, err.cer))
    
    if err.wer < best_wer:
        best_wer = err.wer
        torch.save(SpeechModel.serialize(model), new_model_path)

Epoch 1: 100%|██████████| 29/29 [00:07<00:00,  3.77it/s]


Epoch 1 :: WER: 3.264, CER: 1.140


Epoch 2: 100%|██████████| 29/29 [00:07<00:00,  3.93it/s]


Epoch 2 :: WER: 1.958, CER: 0.865


Epoch 4:   0%|          | 0/29 [00:00<?, ?it/s]

...

Epoch 18 :: WER: 1.697, CER: 0.629


Epoch 20:   0%|          | 0/29 [00:00<?, ?it/s]

Epoch 19 :: WER: 1.305, CER: 0.511


Epoch 20: 100%|██████████| 29/29 [00:07<00:00,  3.74it/s]


Epoch 20 :: WER: 1.567, CER: 0.629
